## Modules

In [1]:
import torch
from tqdm import tqdm
import os
import pandas as pd
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    GenerationConfig,
    LogitsProcessor
)
from peft.tuners.lora import LoraLayer
from torchmetrics.text.rouge import ROUGEScore
from trl import SFTTrainer

## Config

In [2]:
model_path = "/mnt/shared/tibor/Llama-2-13b-chat-hf"
NOT_ANSWERABLE_INCLUDED=False
MAX_NEW_TOKENS = 128
OUTPUT_FILE = "test_llama_13b_4bit_finetuned_all_answerable"
USE_ADAPTER = True
ADAPTER = "./results/results_w_eos_13b/checkpoint-1500"

## Testing

In [3]:
class EosTokenRewardLogitsProcessor(LogitsProcessor):
  def __init__(self,  eos_token_id: int, max_length: int):
    
        if not isinstance(eos_token_id, int) or eos_token_id < 0:
            raise ValueError(f"`eos_token_id` has to be a positive integer, but is {eos_token_id}")

        if not isinstance(max_length, int) or max_length < 1:
          raise ValueError(f"`max_length` has to be a integer bigger than 1, but is {max_length}")

        self.eos_token_id = eos_token_id
        self.max_length=max_length

  def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
    cur_len = input_ids.shape[-1]
    # start to increese the reward of the  eos_tokekn from 80% max length  progressively on length
    for cur_len in (max(0,int(self.max_length*0.8)), self.max_length ):
      ratio = cur_len/self.max_length
      num_tokens = scores.shape[1] # size of vocab
      scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]] =\
      scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]]*ratio*10*torch.exp(-torch.sign(scores[:, [i for i in range(num_tokens) if i != self.eos_token_id]]))
      scores[:, self.eos_token_id] = 1e2*ratio
    return scores

In [4]:
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
#Create a new token and add it to the tokenizer
tokenizer.add_special_tokens({"pad_token":"<pad>"})
tokenizer.padding_side = 'left'

In [5]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="bfloat16",
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_path, quantization_config=bnb_config, device_map={"": 0}
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
#Resize the embeddings
model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=16) # https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc

Embedding(32016, 5120)

In [7]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # Gradient checkpointing is used by default but not compatible with caching

In [8]:
if USE_ADAPTER:
    print("Using " + ADAPTER + " as adapter")
    model = PeftModel.from_pretrained(model, ADAPTER)

Using ./results/results_w_eos_13b/checkpoint-1500 as adapter


In [9]:
impossible_answer = "Nincs elegendő adat a kérdés megválaszolásához."
prompt_prefix = None

if NOT_ANSWERABLE_INCLUDED:
    prompt_prefix = ("Válaszold meg az alábbi kérdést a forrás alapján! "
                     "Ha a forrás alapján nem megválaszolható a kérdés, mondd hogy: "
                     f"{impossible_answer}")
else:
    prompt_prefix = ("Válaszold meg az alábbi kérdést a forrás alapján!")


def generate(context: str, question: str, use_source: bool = True):
    prompt = None
    if use_source:
        prompt = (prompt_prefix
                  + "\n### Forrás:\n" + str(context)
                  + "\n### Kérdés:\n" + str(question)
                  + "\n### Válasz:\n")
    else:
        prompt = ("Válaszold meg az alábbi kérdést"
                 + "\n### Kérdés:\n" + str(question)
                 + "\n### Válasz:\n")

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()

    # print(f"{len(input_ids[0])=}")

    generation_config = GenerationConfig(
        do_sample=True,
        # top_p=1.0,
        # top_k=50,
        # num_beams=1,
        temperature=0.2,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=MAX_NEW_TOKENS)

    generation_output = model.generate(
            # logits_processor=[EosTokenRewardLogitsProcessor(eos_token_id=tokenizer.eos_token_id, max_length = 64)],
            input_ids=input_ids,
            generation_config=generation_config)

    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        return output.split("### Válasz:\n")[1].strip()

In [10]:
test_df = pd.read_csv("data/test.csv", sep=';')
test_df_len = len(test_df)
test_count = int(test_df_len)
# test_count = 10
test_count

483

In [11]:
rouge = ROUGEScore()
r_scores_sum=0
results = {"reference": [],
           "question": [],
           "correct_answer": [],
           "answer_w_ref": [],
           "answer_no_ref": [],
           "rouge1_fmeasure": []}

with tqdm(total=test_count, desc="Running inferences") as pbar:
    for index, row in test_df.iloc[:test_count].iterrows():

        answer_w_ref = generate(context=row['context'], question=row['question'])
        answer_no_ref = generate(context=row['context'], question=row['question'], use_source=False)

        r_score = float(rouge(answer_w_ref, row['answer'])['rouge1_fmeasure'])
        r_scores_sum = r_scores_sum + r_score

        results["reference"].append(row["context"])
        results["question"].append(row["question"])
        results["correct_answer"].append(row["answer"])
        results["answer_w_ref"].append(answer_w_ref)
        results["answer_no_ref"].append(answer_no_ref)
        results["rouge1_fmeasure"].append(r_score)

        pbar.update(1)


Running inferences: 100%|███████████████████████████████████████████████████████████| 483/483 [2:25:47<00:00, 18.11s/it]


In [12]:
print("Average r_score: "+str(r_scores_sum/test_count))

Average r_score: 0.7129637124190419


In [13]:
results_df=pd.DataFrame(results)
results_df.to_csv(OUTPUT_FILE+".csv", sep=';')
results_df.head()

,reference,question,correct_answer,answer_w_ref,answer_no_ref,rouge1_fmeasure
0,A női körülmetélés (vagyis a női nemi szervek ...,Mit jelent a női körülmetélés?,A női körülmetélés (vagyis a női nemi szervek ...,A női körülmetélés (vagyis a női nemi szervek ...,A női körülmetélés (vagy női genitalia megcson...,0.756757
1,Budapest villamosvonal-hálózata a magyar fővár...,Mikor indult meg a lóvasút Budán?,Budán csak 1868. május 18-án avatták fel az el...,Budán csak 1868. május 18-án avatták fel az el...,A lóvasút Budán 1864. április 27-én indult meg...,0.983051
2,"Apja testvérének fiai, azaz István unokatestvé...",Volt testvére Orseolo Péternek?,Választása végül Ilona nevű lánytestvérének fi...,Választása végül Ilona nevű lánytestvérének fi...,"Orseolo Péternek volt egy testvére, Orseolo Ot...",0.992126
3,A globális felmelegedés következtében a gleccs...,A gleccserek édes vagy sós vizet tartalmaznak?,A globális felmelegedés következtében a gleccs...,A globális felmelegedés következtében a gleccs...,"A gleccserek édes víztartalmazza, amelyet a hó...",0.979592
4,Az átlagos egyiptomi lakóépületek a falvakban ...,Min aludtak az ókori egyiptomiak?,"Szalmazsákon aludtak, párna helyett kőből vagy...","Szalmazsákon aludtak, párna helyett kőből vagy...",A 2. és 3. életkorban a gyermekek a gyakorlatb...,0.980392


In [14]:
tokenizer.eos_token_id


2